In [1]:
from fosforml.model_manager.snowflakesession import get_session
session = get_session()

/opt/conda/lib/python3.9/site-packages/fosforml-1.2.1.2-py3.9.egg/fosforml/model_manager/model_registry.py:101: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/opt/conda/lib/python3.9/site-packages/fosforml-1.2.1.2-py3.9.egg/fosforml/model_manager/model_registry.py:174: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/opt/conda/lib/python3.9/site-packages/fosforml-1.2.1.2-py3.9.egg/fosforml/model_manager/model_registry.py:101: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/opt/conda/lib/python3.9/site-packages/fosforml-1.2.1.2-py3.9.egg/fosforml/model_manager/model_registry.py:174: SyntaxWarning: "is not" with a literal. Did you mean "!="?
2025-03-05 07:09:02,199 - fdc_connector_python_sdk.connectors.snowflake - INFO - starting the snowpark session creation
2025-03-05 07:09:43,499 - fdc_connector_python_sdk.connectors.snowflake - INFO - time taken to create snowflake session 41.30029892921448


In [2]:
import pandas as pd
import numpy as np
from fosforml import register_model

In [3]:
# load the iris dataset as an example 
from sklearn.datasets import load_iris 
iris= load_iris() 
iris.keys()

dict_keys=(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [4]:
iris= pd.DataFrame(
    data = np.c_[iris['data'], iris['target']],
    columns= iris['feature_names'] + ['target']
    )
iris.columns = ['SEPAL_LENGTH', 'SEPAL_WIDTH', 'PETAL_LENGTH', 'PETAL_WIDTH', 'TARGET']
iris.head(10)

SEPAL_LENGTH  SEPAL_WIDTH  PETAL_LENGTH  PETAL_WIDTH  TARGET
0           5.1          3.5           1.4          0.2     0.0
1           4.9          3.0           1.4          0.2     0.0
2           4.7          3.2           1.3          0.2     0.0
3           4.6          3.1           1.5          0.2     0.0
4           5.0          3.6           1.4          0.2     0.0
5           5.4          3.9           1.7          0.4     0.0
6           4.6          3.4           1.4          0.3     0.0
7           5.0          3.4           1.5          0.2     0.0
8           4.4          2.9           1.4          0.2     0.0
9           4.9          3.1           1.5          0.1     0.0

In [5]:
species = []

for i in range(len(iris['TARGET'])):
    if iris['TARGET'][i] == 0:
        species.append("setosa")
    elif iris['TARGET'][i] == 1:
        species.append('versicolor')
    else:
        species.append('virginica')


iris['SPECIES'] = species

iris

SEPAL_LENGTH  SEPAL_WIDTH  PETAL_LENGTH  PETAL_WIDTH  TARGET    SPECIES
0             5.1          3.5           1.4          0.2     0.0     setosa
1             4.9          3.0           1.4          0.2     0.0     setosa
2             4.7          3.2           1.3          0.2     0.0     setosa
3             4.6          3.1           1.5          0.2     0.0     setosa
4             5.0          3.6           1.4          0.2     0.0     setosa
..            ...          ...           ...          ...     ...        ...
145           6.7          3.0           5.2          2.3     2.0  virginica
146           6.3          2.5           5.0          1.9     2.0  virginica
147           6.5          3.0           5.2          2.0     2.0  virginica
148           6.2          3.4           5.4          2.3     2.0  virginica
149           5.9          3.0           5.1          1.8     2.0  virginica

[150 rows x 6 columns]

In [6]:
from sklearn.model_selection import train_test_split

# Droping the target and species since we only need the measurements
X = iris.drop(['TARGET','SPECIES'], axis=1)

# converting into numpy array and assigning petal length and petal width
# X = X.to_numpy()[:, (2,3)]
y = iris['TARGET']

# Splitting into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5, random_state=42)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
# Initialize and train the GradientBoosting classifier
model = GradientBoostingClassifier()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

In [ ]:
y_train = y_train.to_frame()
y_test = y_test.to_frame()
y_pred = pd.DataFrame(y_pred, columns=["PREDICTION"])

In [9]:
register_model(
    model_obj=model, 
    session=session,
    x_train=X_train,
    y_train=y_train,
    x_test=X_test,
    y_test=y_test,
    y_pred=y_pred,
    source="Notebook",
    dataset_name="IRIS_DATASET",
    dataset_source="Snowflake",
    name="XGB_Classifier_test_02",
    description=" Classification Model",
    flavour="sklearn",
    model_type="classification",
    conda_dependencies=["scikit-learn"]
)

Starting model registration via fosforml


Identified model object type: <class 'sklearn.ensemble._gb.GradientBoostingClassifier'>
Hyperparameters: {'ccp_alpha': '0.0', 'criterion': 'friedman_mse', 'init': None, 'learning_rate': '0.1', 'loss': 'log_loss', 'max_depth': '3', 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': '0.0', 'min_samples_leaf': '1', 'min_samples_split': '2', 'min_weight_fraction_leaf': '0.0', 'n_estimators': '100', 'n_iter_no_change': None, 'random_state': None, 'subsample': '1.0', 'tol': '0.0001', 'validation_fraction': '0.1', 'verbose': '0', 'warm_start': 'False'}


KeyboardInterrupt: 